In [9]:
from helpers import *

import shutil


import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


import datetime
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


from collections import OrderedDict
from itertools import product



linestyles = OrderedDict(
    [

     ('loosely dashed',      (0, (5, 20))),
     ('dashed',              (0, (3, 2))),
     ('densely dashed',      (0, (1, 3))),

    ])

# Data Processing

## Importing data

In [10]:
def load_kaggle_data(path):
    data = pd.read_csv(path)
    data.replace({'\\N':np.nan}, inplace=True)
    return data

data_dir ='kaggle-data/'

circuits_raw_df = load_kaggle_data(f'{data_dir}/circuits.csv')
constructors_raw_df = load_kaggle_data(f'{data_dir}/constructors.csv')
constructor_standings_raw_df = load_kaggle_data(f'{data_dir}/constructor_standings.csv')
driver_standings_raw_df = load_kaggle_data(f'{data_dir}/driver_standings.csv')
drivers_raw_df = load_kaggle_data(f'{data_dir}/drivers.csv')
laptimes_raw_df = load_kaggle_data(f'{data_dir}/lap_times.csv')
pitstops_raw_df = load_kaggle_data(f'{data_dir}/pit_stops.csv')
qualifying_raw_df = load_kaggle_data(f'{data_dir}/qualifying.csv')
races_raw_df = load_kaggle_data(f'{data_dir}/races.csv')
results_raw_df = load_kaggle_data(f'{data_dir}/results.csv')
seasons_raw_df = load_kaggle_data(f'{data_dir}/seasons.csv')
sprint_results = load_kaggle_data(f'{data_dir}/sprint_results.csv')
status_raw_df = load_kaggle_data(f'{data_dir}/status.csv')

print('circuits_raw_df', circuits_raw_df.columns)
print('constructors_raw_df', constructors_raw_df.columns)
print('constructor_standings_raw_df', constructor_standings_raw_df.columns)
print('driver_standings_raw_df', driver_standings_raw_df.columns)
print('drivers_raw_df', drivers_raw_df.columns)
print('laptimes_raw_df', laptimes_raw_df.columns)
print('pitstops_raw_df', pitstops_raw_df.columns)
print('qualifying_raw_df', qualifying_raw_df.columns)
print('races_raw_df', races_raw_df.columns)
print('results_raw_df', results_raw_df.columns)
print('seasons_raw_df', seasons_raw_df.columns)
print('sprint_results', sprint_results.columns)
print('status_raw_df', status_raw_df.columns)

circuits_raw_df Index(['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng',
       'alt', 'url'],
      dtype='object')
constructors_raw_df Index(['constructorId', 'constructorRef', 'name', 'nationality', 'url'], dtype='object')
constructor_standings_raw_df Index(['constructorStandingsId', 'raceId', 'constructorId', 'points',
       'position', 'positionText', 'wins'],
      dtype='object')
driver_standings_raw_df Index(['driverStandingsId', 'raceId', 'driverId', 'points', 'position',
       'positionText', 'wins'],
      dtype='object')
drivers_raw_df Index(['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob',
       'nationality', 'url'],
      dtype='object')
laptimes_raw_df Index(['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds'], dtype='object')
pitstops_raw_df Index(['raceId', 'driverId', 'stop', 'lap', 'time', 'duration',
       'milliseconds'],
      dtype='object')
qualifying_raw_df Index(['qualifyId', 'raceId', 'driverId',

## Merging data
Note: not filtering for drivers

In [11]:
# # Process drivers data set

# drivers = drivers_raw_df[['driverId','forename','surname','number']].reset_index(drop=True)
# # Make name column
# drivers['Name'] = drivers[['forename', 'surname']].agg(' '.join, axis=1)
# drivers.drop(['forename','surname'], axis=1, inplace=True)

# driver_name_id_map = dict(zip(drivers['Name'], drivers['driverId']))
# def get_driverID(name):
#     return drivers.loc[drivers['Name']==name, 'driverId'].squeeze()

# driver_name_number_map = dict(zip(drivers['Name'], drivers['number']))
# def get_driver_number(name):
#     return drivers.loc[drivers['Name']==name, 'number'].squeeze()

# driver_names_of_interest = ['Max Verstappen',
#                            'Sergio Pérez',
#                            'Lewis Hamilton',
#                            'Fernando Alonso',
#                            'Charles Leclerc',
#                            'Lando Norris',
#                            'George Russell',
#                            'Oscar Piastri',
#                            'Lance Stroll',
#                            'Pierre Gasly',
#                            'Esteban Ocon',
#                            'Alexander Albon',
#                            'Nico Hülkenberg',
#                            'Valtteri Bottas',
#                            'Guanyu Zhou',
#                            'Yuki Tsunoda',
#                            'Kevin Magnussen',
#                            #'Liam Lawson',
#                            'Logan Sargeant',
#                            'Nyck de Vries',
#                            'Daniel Ricciardo']
# driver_ids_of_interest = [driver_name_id_map[name] for name in driver_names_of_interest]

# driver_numbers_of_interest = [driver_name_number_map[name] for name in driver_names_of_interest]


In [12]:
# Driver name
drivers = drivers_raw_df[['driverId','forename','surname','number']].reset_index(drop=True)
drivers['Name'] = drivers[['forename', 'surname']].agg(' '.join, axis=1)
drivers.drop(['forename','surname'], axis=1, inplace=True)


# start with race results
# extend positionText labels (see Appendix)
results_raw_df['positionText'] = results_raw_df['positionText'].replace({'R':'Retired','D':'Disqualified','N':'not classified', 'E':'Excluded','W':'Withdrawn','F':'failed to qualify'})
# drop time, milliseconds, fastestLap, rank, fastestLapTime, fastestLapSpeed, and number (http://ergast.com/mrd/methods/results/)
races = results_raw_df.drop(['time','milliseconds','fastestLap','rank','fastestLapTime','fastestLapSpeed','number'], axis=1)
races.drop(['positionOrder'], axis=1, inplace=True)
races['position'].fillna('NaN', inplace=True)
#display(races.isna().sum())
assert races.isna().sum().sum() == 0, 'there are null values'

# Get race data (ignoring qualifying data)
races_raw_df.rename(columns={'name':'Race'}, inplace=True)
prev_len = races.shape[0]
races = races.merge(races_raw_df[['raceId','Race','round','circuitId','date','url']], on='raceId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races.isna().sum().sum() == 0

# Explode date
races['date'] = pd.to_datetime(races['date'])
races['Month'] = races['date'].dt.month
races['Year'] = races['date'].dt.year
#races.drop(['date'], axis=1, inplace=True)

# Filter year due to bad quality data in early years (multiple results reported for a driver in a race) -- see EDA for details
races = races[races['Year']>1964].reset_index(drop=True)
assert races.groupby(['raceId','driverId'])['position'].nunique().max()==1, 'there are multiple positions reported for a driver in a race'

# Merge with drivers
prev_len = races.shape[0]
races = races.merge(drivers[['driverId','Name']], on='driverId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races['Name'].isna().sum() == 0


# # Filter for drivers of interest
# races = races[races['Name'].isin(driver_names_of_interest)].reset_index(drop=True)
# assert races['driverId'].nunique() == len(driver_ids_of_interest), 'confirm we have lap data for all drivers of interest'
# #races.drop(['driverId'], axis=1, inplace=True)
# assert races.groupby(['raceId','Name']).size().max() == 1, 'driver occurs twice in a race'
# assert races.groupby(['raceId','driverId'])['position'].nunique().max() == 1


# Get status string
prev_len = races.shape[0]
races = races.merge(status_raw_df[['statusId','status']], on='statusId', how='left')
assert races['status'].isna().sum() == 0
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
races.drop(['statusId'], axis=1, inplace=True)


# Get car name
prev_len = races.shape[0]
constructors_raw_df.rename(columns={'name':'Car'}, inplace=True)
car_names = constructors_raw_df['Car'].unique()
races = races.merge(constructors_raw_df[['constructorId','Car']], on='constructorId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races['Car'].isna().sum() == 0
races.drop(['constructorId'], axis=1, inplace=True)


# Get circuit data
prev_len = races.shape[0]
circuits_raw_df.rename(columns={'name':'Circuit'}, inplace=True)
circuits_raw_df['alt'].fillna(circuits_raw_df['alt'].median(), inplace=True) # impute median altitude -- #todo may want to change
races = races.merge(circuits_raw_df[['circuitId','Circuit','location','country','alt','lat','lng']], on='circuitId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races.isna().sum().sum() == 0
races.drop(['circuitId'], axis=1, inplace=True)

# Get number of laps in the race
number_of_laps = results_raw_df[results_raw_df['statusId']==1].groupby(['raceId'])['laps'].max().reset_index(name='nlaps')
assert races.isna().sum().sum() == 0, 'there is at least one race where none of the drivers finished -- cannot find number of laps in the race'
prev_len = races.shape[0]
races = races.merge(number_of_laps, on='raceId',how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races.isna().sum().sum() == 0, 'there is at least one race where none of the drivers finished -- cannot find number of laps in the race'
races.drop(['laps'], axis=1, inplace=True)



# Summary
print('First year:',  races['Year'].min())
print('# Races:', races['raceId'].nunique())

First year: 1965
# Races: 960


In [5]:
# # Race Model Filters

# # filter for races with 32 or fewer drivers
# prev_n_races = races['raceId'].nunique()

# valid_races = races.groupby('raceId').size()
# valid_races = valid_races[valid_races<=32].index
# races = races[races['raceId'].isin(valid_races)].reset_index(drop=True)
# print('Races removed:', prev_n_races - races['raceId'].nunique(), '({} races now)'.format(races['raceId'].nunique()))



In [13]:
## Dynamic race model filters

# when number of race laps < k + threshold
k = 10
threshold = k*2
prev_n_races = races['raceId'].nunique()
race_counts = races['raceId'].value_counts()
valid_races = race_counts[race_counts > 10].index
races = races[races['raceId'].isin(valid_races)].reset_index(drop=True)
print('Races removed:', prev_n_races - races['raceId'].nunique())

Races removed: 0


In [14]:
print('unique races', races['raceId'].nunique())

unique races 960


## Construct Model Data With Lap Data

In [72]:
# adds padding rows to races with less than 24 drivers
def pad_race(group, target_rows=24, invalid_finishing_position_num=25):
    current_rows = group.shape[0]
    # If the current number of rows is less than the target, pad the group
    if current_rows < target_rows:
        # Calculate the number of rows to append
        rows_to_add = target_rows - current_rows
        # Create a DataFrame with the same columns, filled with NaNs
        padding_df = pd.DataFrame(np.nan, index=range(rows_to_add), columns=group.columns)
        # Assign the raceId to the padding DataFrame
        padding_df['raceId'] = group['raceId'].iloc[0]
        padding_df['driverId'] = -1
        padding_df['y'] = invalid_finishing_position_num
        if 'coef' in padding_df.columns:
            padding_df['coef'] = group['coef'].max() + group['coef'].std()
            #print('padding_df', padding_df['coef'].max())

        # Append the padding DataFrame to the group
        group = pd.concat([group, padding_df], ignore_index=True)
    return group

# Replace race driver order with random order with num_copies copies
def randomize_race(group, num_copies=4):
    randomized_races = []
    race_id = group['raceId'].iloc[0]

    # Create each random copy
    for i in range(num_copies):
        randomized_group = group.sample(frac=1, replace=False, random_state=i+1).reset_index(drop=True)
        randomized_group['copy_id'] = f"{race_id}_{i}"  # Create a unique ID for each copy
        randomized_races.append(randomized_group)
    
    # Concatenate
    return pd.concat(randomized_races, ignore_index=True)

def train_test_split_by_id(df, id_column, train_size=0.8):
    unique_ids = df[id_column].unique()
    
    train_ids, temp_ids = train_test_split(unique_ids, train_size=train_size, random_state=100)

    test_ids, valid_ids = train_test_split(temp_ids, train_size=0.5, random_state=100)

    train_df = df[df[id_column].isin(train_ids)]
    valid_df = df[df[id_column].isin(valid_ids)]
    test_df = df[df[id_column].isin(test_ids)]
    
    return train_df, valid_df, test_df

def pivot_model_data(df):

    # value columns
    laps_and_y_cols = [col for col in df.columns if 'lap_' in col or col == 'y' or col == 'coef']

    pivot = df.pivot_table(index='copy_id', 
                        columns=['model_driverId'], 
                        values=laps_and_y_cols, fill_value=np.nan)

    # Flatten the MultiIndex in columns
    #pivot.columns = [f'model_driver{model_driver_id}_{lap_id}pos' if lap_id else 'copy_id' for model_driver_id, lap_id in pivot.columns]
    new_columns = []
    for col in pivot.columns.tolist()[:]:
        # Concatenate the model_driverId with the lap information
        driver_number = 'driver' + str(col[1])  # Increment model_driverId by 1 to start numbering from 1
        lap_info = col[0]
        new_columns.append(f'{driver_number}_{lap_info}')
    pivot.columns = new_columns
    pivot.reset_index(inplace=True, drop=False)
    return pivot

In [88]:
def make_model_data(laptimes_raw_df, races, laps_to_consider, laps_text, path, coef=False, race_copies=3, metric='milliseconds'):
    # get laptimes (note: all races in laptimes_raw_df are in races dataframe)
    laptimes = laptimes_raw_df[laptimes_raw_df['raceId'].isin(races['raceId'].unique())].reset_index(drop=True)[['raceId','driverId','lap','position','milliseconds']]
    n_drivers = laptimes.groupby(['raceId'])['driverId'].nunique().max()
    print('max number drivers per race', n_drivers)

    # calculate driver finishing times
    driver_finishing_times = laptimes.groupby(['raceId','driverId'])['milliseconds'].sum().reset_index(name='finishing_time') # todo: fix where driver doesn't finish

    # filter data so that it corresponds to desired feature set
    laptimes = laptimes[laptimes['lap'].isin(laps_to_consider)].reset_index(drop=True)

    laptimes['lap'] = 'lap_' + laptimes['lap'].astype(str).str.zfill(2)
    #print(laptimes['lap'].unique())

    # pivot lap data
    if metric == 'milliseconds':
        model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=0)  # fill missing values with 0
    elif metric == 'position':
        model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=25)  # fill missing values with 25
    else:
        raise ValueError('metric must be milliseconds or position')

    model_data.reset_index(inplace=True)

    driver_feature = pd.read_csv('data/driver_feature_model3_coef.csv')
    model_data = model_data.merge(driver_feature[['driverId','coef']], on=['driverId'], how='left')
    model_data['coef'] = model_data['coef'].fillna(model_data['coef'].quantile(0.95)) # these are drivers that don't have a coef due to only being in 1 race -- impute with 95th percentile value

    # Finishing positions (Y)
    positions = races[['raceId','driverId','positionText']].drop_duplicates().reset_index(drop=True)
    model_position_text_map = {'Retired':'25','Disqualified':'25', 'not classified':'25','Excluded':'25'}

    positions['y'] = positions['positionText'].replace(model_position_text_map)
    positions.drop(['positionText'], axis=1, inplace=True)
    prev_len = model_data.shape[0]
    model_data = model_data.merge(positions, on=['raceId','driverId'], how='left')
    #display(model_data['Y'].value_counts())

    assert model_data.shape[0] == prev_len, 'duplicates formed during merge'
    assert model_data.groupby(['raceId','driverId'])['y'].nunique().max() == 1, 'each driver should have one finishing position'
    assert model_data['y'].isin(['Withdrawn','failed to qualify']).sum() == 0, 'should not have any drivers who withdrew or failed to qualify'
    model_data['y'] = model_data['y'].astype(int)
    assert model_data.groupby(['raceId'])['y'].min().max() == 1, 'races missing a winner'
    assert model_data.groupby(['raceId'])['y'].min().min() == 1, 'positions < 1'
    assert model_data.groupby(['raceId'])['y'].max().max() <= 25, 'race has position > 24'
    assert model_data.isna().sum().sum() ==0

    assert model_data.isna().sum().sum() == 0, 'there are null values'
    # add rows if number of drivers in a race is less than 24
    model_data = model_data.groupby('raceId').apply(pad_race).reset_index(drop=True)
    if metric == 'milliseconds':
        model_data.fillna(0, inplace=True)
    else:
        model_data.fillna(25, inplace=True)
        assert model_data['y'].min() == 1, 'positions < 1'

    

     # replace data with randomly ordered copies
    model_data = model_data.groupby('raceId', group_keys=False).apply(randomize_race, (race_copies)).reset_index(drop=True)
    # Create 'model_driverId' within each 'copy_id' group
    model_data['model_driverId'] = model_data.groupby('copy_id').cumcount() + 1

    train_df, valid_df, test_df = train_test_split_by_id(model_data, 'copy_id')

    train_df_mf = pivot_model_data(train_df).set_index('copy_id')
    valid_df_mf = pivot_model_data(valid_df).set_index('copy_id')
    test_df_mf = pivot_model_data(test_df).set_index('copy_id')

    n_features = n_drivers * len(laps_to_consider)
    if coef:
        n_features += n_drivers

    train_X, train_y = train_df_mf.iloc[:,:n_features], train_df_mf.iloc[:,n_features:]
    valid_X, valid_y = valid_df_mf.iloc[:,:n_features], valid_df_mf.iloc[:,n_features:]
    test_X, test_y = test_df_mf.iloc[:,:n_features], test_df_mf.iloc[:,n_features:]
    
    coef_text = 'coef' if coef else 'no_coef'

    #binary_str = 'binary' if binary_y else 'categorical'
    path = '{}/{}/{}/{}/'.format(path, metric, coef_text, laps_text)

    # make folder if doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)

    # print(train_X.shape, train_y.shape)
    # print(valid_X.shape, valid_y.shape)
    # print(test_X.shape, test_y.shape)
    
    train_X.to_csv('{}{}copies_X_train.csv'.format(path, race_copies), index=True)
    train_y.to_csv('{}{}copies_y_train.csv'.format(path, race_copies), index=True)
    valid_X.to_csv('{}{}copies_X_valid.csv'.format(path, race_copies), index=True)
    valid_y.to_csv('{}{}copies_y_valid.csv'.format(path, race_copies), index=True)
    test_X.to_csv('{}{}copies_X_test.csv'.format(path, race_copies), index=True)
    test_y.to_csv('{}{}copies_y_test.csv'.format(path, race_copies), index=True)

In [90]:
path = 'Data/RaceMultiOutputModelRandomized2/'

# delete everything in path
for root, dirs, files in os.walk(path):
    for file in files:
        os.remove(os.path.join(root, file))
    for dir in dirs:
        shutil.rmtree(os.path.join(root, dir))

# Baseline datasets
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[1], laps_text='Lap1',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[2], laps_text='Lap2',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[3], laps_text='Lap3',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[4], laps_text='Lap4',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[5], laps_text='Lap5',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[6], laps_text='Lap6',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[7], laps_text='Lap7',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[8], laps_text='Lap8',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[9], laps_text='Lap9',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[10], laps_text='Lap10',
                race_copies=1, metric='position', path=path)



# Laps0to10 datasets
for metric in ['milliseconds', 'position']:
    for coef in [True, False]:
    

        make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[1], laps_text='Lap1',
                coef=coef, race_copies=3, metric=metric, path=path)
        make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[10], laps_text='Lap10',
                coef=coef, race_copies=3, metric=metric, path=path)
        
        make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[1,2,3,4,5,6,7,8,9,10], laps_text='Laps0to10',
                        coef=coef, race_copies=3, metric=metric, path=path)
    
notify('done building datasets','')

max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24


# inline driver feature

In [83]:
laps_to_consider

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [78]:
laps_to_consider = list(range(1,11))
laps_text = 'Laps1to2'
path = 'Data/RaceMultiOutputModelRandomized/'
race_copies = 1
metric = 'position'

def make_model_data(laptimes_raw_df, races, laps_to_consider, laps_text, path, race_copies=3, metric='milliseconds'):
    # get laptimes (note: all races in laptimes_raw_df are in races dataframe)
    laptimes = laptimes_raw_df[laptimes_raw_df['raceId'].isin(races['raceId'].unique())].reset_index(drop=True)[['raceId','driverId','lap','position','milliseconds']]
    n_drivers = laptimes.groupby(['raceId'])['driverId'].nunique().max()
    print('max number drivers per race', n_drivers)

    # calculate driver finishing times
    driver_finishing_times = laptimes.groupby(['raceId','driverId'])['milliseconds'].sum().reset_index(name='finishing_time') # todo: fix where driver doesn't finish

    # filter data so that it corresponds to desired feature set
    laptimes = laptimes[laptimes['lap'].isin(laps_to_consider)].reset_index(drop=True)

    laptimes['lap'] = 'lap_' + laptimes['lap'].astype(str).str.zfill(2)
    #print(laptimes['lap'].unique())

    # pivot lap data
    if metric == 'milliseconds':
        model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=0)  # fill missing values with 0
    elif metric == 'position':
        model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=25)  # fill missing values with 25
    else:
        raise ValueError('metric must be milliseconds or position')

    model_data.reset_index(inplace=True)

    driver_feature = pd.read_csv('data/driver_feature_model3_coef.csv')
    model_data = model_data.merge(driver_feature[['driverId','coef']], on=['driverId'], how='left')
    model_data['coef'] = model_data['coef'].fillna(model_data['coef'].quantile(0.95)) # these are drivers that don't have a coef due to only being in 1 race -- impute with 95th percentile value


    # Finishing positions (Y)
    positions = races[['raceId','driverId','positionText']].drop_duplicates().reset_index(drop=True)
    model_position_text_map = {'Retired':'25','Disqualified':'25', 'not classified':'25','Excluded':'25'}

    positions['y'] = positions['positionText'].replace(model_position_text_map)
    positions.drop(['positionText'], axis=1, inplace=True)
    prev_len = model_data.shape[0]
    model_data = model_data.merge(positions, on=['raceId','driverId'], how='left')
    #display(model_data['Y'].value_counts())

    assert model_data.shape[0] == prev_len, 'duplicates formed during merge'
    assert model_data.groupby(['raceId','driverId'])['y'].nunique().max() == 1, 'each driver should have one finishing position'
    assert model_data['y'].isin(['Withdrawn','failed to qualify']).sum() == 0, 'should not have any drivers who withdrew or failed to qualify'
    model_data['y'] = model_data['y'].astype(int)
    assert model_data.groupby(['raceId'])['y'].min().max() == 1, 'races missing a winner'
    assert model_data.groupby(['raceId'])['y'].min().min() == 1, 'positions < 1'
    assert model_data.groupby(['raceId'])['y'].max().max() <= 25, 'race has position > 24'
    assert model_data.isna().sum().sum() ==0

    assert model_data.isna().sum().sum() == 0, 'there are null values'
    # add rows if number of drivers in a race is less than 24
    model_data = model_data.groupby('raceId').apply(pad_race).reset_index(drop=True)
    if metric == 'milliseconds':
        model_data.fillna(0, inplace=True)
    else:
        model_data.fillna(25, inplace=True)
        assert model_data['y'].min() == 1, 'positions < 1'



        # replace data with randomly ordered copies
    model_data = model_data.groupby('raceId', group_keys=False).apply(randomize_race, (race_copies)).reset_index(drop=True)
    # Create 'model_driverId' within each 'copy_id' group
    model_data['model_driverId'] = model_data.groupby('copy_id').cumcount() + 1


    train_df, valid_df, test_df = train_test_split_by_id(model_data, 'copy_id')

    train_df_mf = pivot_model_data(train_df).set_index('copy_id')
    valid_df_mf = pivot_model_data(valid_df).set_index('copy_id')
    test_df_mf = pivot_model_data(test_df).set_index('copy_id')

    coef= True
    n_features = n_drivers * len(laps_to_consider)
    if coef:
        n_features += n_drivers

    train_X, train_y = train_df_mf.iloc[:,:n_features], train_df_mf.iloc[:,n_features:]
    valid_X, valid_y = valid_df_mf.iloc[:,:n_features], valid_df_mf.iloc[:,n_features:]
    test_X, test_y = test_df_mf.iloc[:,:n_features], test_df_mf.iloc[:,n_features:]


    # #binary_str = 'binary' if binary_y else 'categorical'
    # path = '{}/{}/{}/'.format(path, metric, laps_text)

    # # make folder if doesn't exist
    # if not os.path.exists(path):
    #     os.makedirs(path)

    # # print(train_X.shape, train_y.shape)
    # # print(valid_X.shape, valid_y.shape)
    # # print(test_X.shape, test_y.shape)

    # train_X.to_csv('{}{}copies_X_train.csv'.format(path, race_copies), index=True)
    # train_y.to_csv('{}{}copies_y_train.csv'.format(path, race_copies), index=True)
    # valid_X.to_csv('{}{}copies_X_valid.csv'.format(path, race_copies), index=True)
    # valid_y.to_csv('{}{}copies_y_valid.csv'.format(path, race_copies), index=True)
    # test_X.to_csv('{}{}copies_X_test.csv'.format(path, race_copies), index=True)
    # test_y.to_csv('{}{}copies_y_test.csv'.format(path, race_copies), index=True)

max number drivers per race 24


In [84]:

train_df, valid_df, test_df = train_test_split_by_id(model_data, 'copy_id')

train_df_mf = pivot_model_data(train_df).set_index('copy_id')
valid_df_mf = pivot_model_data(valid_df).set_index('copy_id')
test_df_mf = pivot_model_data(test_df).set_index('copy_id')

coef= True
n_features = n_drivers * len(laps_to_consider)
if coef:
    n_features += n_drivers

train_X, train_y = train_df_mf.iloc[:,:n_features], train_df_mf.iloc[:,n_features:]
valid_X, valid_y = valid_df_mf.iloc[:,:n_features], valid_df_mf.iloc[:,n_features:]
test_X, test_y = test_df_mf.iloc[:,:n_features], test_df_mf.iloc[:,n_features:]


# #binary_str = 'binary' if binary_y else 'categorical'
# path = '{}/{}/{}/'.format(path, metric, laps_text)

# # make folder if doesn't exist
# if not os.path.exists(path):
#     os.makedirs(path)

# # print(train_X.shape, train_y.shape)
# # print(valid_X.shape, valid_y.shape)
# # print(test_X.shape, test_y.shape)

# train_X.to_csv('{}{}copies_X_train.csv'.format(path, race_copies), index=True)
# train_y.to_csv('{}{}copies_y_train.csv'.format(path, race_copies), index=True)
# valid_X.to_csv('{}{}copies_X_valid.csv'.format(path, race_copies), index=True)
# valid_y.to_csv('{}{}copies_y_valid.csv'.format(path, race_copies), index=True)
# test_X.to_csv('{}{}copies_X_test.csv'.format(path, race_copies), index=True)
# test_y.to_csv('{}{}copies_y_test.csv'.format(path, race_copies), index=True)

In [86]:
train_X.shape

(408, 264)

In [39]:
races[races['driverId']==827]

,resultId,raceId,driverId,grid,position,positionText,points,Race,round,date,url,Month,Year,Name,status,Car,Circuit,location,country,alt,lat,lng,nlaps
19258,22394,911,827,21,NaN,Retired,0.0,Belgian Grand Prix,12,2014-08-24,http://en.wikipedia.org/wiki/2014_Belgian_Gran...,8,2014,André Lotterer,Power Unit,Caterham,Circuit de Spa-Francorchamps,Spa,Belgium,401,50.4372,5.97139,44


In [36]:
driver_feature[driver_feature['driverId']==28]

,driverId,coef,scaled_coef


In [35]:
model_data[model_data['coef'].isna()]

,raceId,driverId,lap_01,lap_02,coef,y
887,45,28,10,1,NaN,25
3514,176,72,15,20,NaN,25
6806,911,827,18,25,NaN,25


In [34]:
for c in model_data.columns:
    print(c, model_data[c].isna().sum())

raceId 0
driverId 0
lap_01 0
lap_02 0
coef 3
y 0


# testing

In [319]:
train_y

,driver1_y,driver2_y,driver3_y,driver4_y,driver5_y,driver6_y,driver7_y,driver8_y,driver9_y,driver10_y,driver11_y,driver12_y,driver13_y,driver14_y,driver15_y,driver16_y,driver17_y,driver18_y,driver19_y,driver20_y,driver21_y,driver22_y,driver23_y,driver24_y
copy_id,,,,,,,,,,,,,,,,,,,,,,,,
1000_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1000_3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1001_1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1001_2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99_1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99_2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
99_3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [300]:
train_df[train_df['copy_id']=='1000_1']

,raceId,driverId,lap_01,lap_02,lap_03,lap_04,lap_05,lap_06,lap_07,lap_08,lap_09,lap_10,y,copy_id,model_driverId
29232,1000,830,5,5,5,5,5,25,25,25,25,25,0,1000_1,1
29233,1000,828,18,18,18,18,18,18,18,18,18,18,0,1000_1,2
29234,1000,0,0,0,0,0,0,0,0,0,0,0,0,1000_1,3
29235,1000,839,14,14,14,15,15,14,14,14,14,14,0,1000_1,4
29236,1000,0,0,0,0,0,0,0,0,0,0,0,0,1000_1,5
29237,1000,4,11,11,11,11,11,10,11,11,12,12,0,1000_1,6
29238,1000,838,13,13,13,13,13,13,13,13,13,13,0,1000_1,7
29239,1000,0,0,0,0,0,0,0,0,0,0,0,0,1000_1,8
29240,1000,842,6,6,6,6,6,5,5,5,5,5,0,1000_1,9
29241,1000,822,2,2,2,2,2,2,2,2,2,2,0,1000_1,10


In [290]:
pt = train_df[train_df['copy_id'].isin(['1000_1','1000_3','9_2'])]
pt = pt[pt['model_driverId']<=4]
pt

,raceId,driverId,lap_01,lap_02,lap_03,lap_04,lap_05,lap_06,lap_07,lap_08,lap_09,lap_10,y,copy_id,model_driverId
600,9,16,8,8,8,8,8,8,8,8,8,8,0,9_2,1
601,9,22,1,1,1,1,1,1,1,1,1,1,0,9_2,2
602,9,4,11,11,11,11,11,11,11,11,11,11,0,9_2,3
603,9,17,2,2,2,2,2,2,2,2,2,2,1,9_2,4
29232,1000,830,5,5,5,5,5,25,25,25,25,25,0,1000_1,1
29233,1000,828,18,18,18,18,18,18,18,18,18,18,0,1000_1,2
29234,1000,0,0,0,0,0,0,0,0,0,0,0,0,1000_1,3
29235,1000,839,14,14,14,15,15,14,14,14,14,14,0,1000_1,4
29280,1000,0,0,0,0,0,0,0,0,0,0,0,0,1000_3,1
29281,1000,1,1,1,1,1,1,1,1,1,1,1,1,1000_3,2


In [291]:
ptr = pivot_model_data(pt)
ptr

copy_id lap_01             lap_02             lap_03          \
model_driverId              1   2   3   4      1   2   3   4      1   2   3   
0               1000_1      5  18   0  14      5  18   0  14      5  18   0   
1               1000_3      0   1   0  15      0   1   0  15      0   1   0   
2                  9_2      8   1  11   2      8   1  11   2      8   1  11   

                   lap_04             lap_05             lap_06              \
model_driverId   4      1   2   3   4      1   2   3   4      1   2   3   4   
0               14      5  18   0  15      5  18   0  15     25  18   0  14   
1               16      0   1   0  16      0   1   0  16      0   1   0  15   
2                2      8   1  11   2      8   1  11   2      8   1  11   2   

               lap_07             lap_08             lap_09              \
model_driverId      1   2   3   4      1   2   3   4      1   2   3   4   
0                  25  18   0  14     25  18   0  14     25  18   0  14   
1                   0   1   0  15      0   1   0  15      0   1   0  15   
2                   8   1  11   2      8   1  11   2      8   1  11   2   

               lap_10              y           
model_driverId      1   2   3   4  1  2  3  4  
0                  25  18   0  14  0  0  0  0  
1                   0   1   0  15  0  1  0  0  
2                   8   1  11   2  0  0  0  1

In [304]:
new_columns = ['copy_id']
for col in ptr.columns.tolist()[1:]:
    # Concatenate the model_driverId with the lap information
    # Assuming model_driverId is represented by numbers starting from 1
    # And the lap information is something like 'lap_01', 'lap_02', etc.
    driver_number = 'driver' + str(col[1])  # Increment model_driverId by 1 to start numbering from 1
    lap_info = col[0]
    new_columns.append(f'{driver_number}_{lap_info}')
new_columns

['driver1_lap_01',
 'driver2_lap_01',
 'driver3_lap_01',
 'driver4_lap_01',
 'driver1_lap_02',
 'driver2_lap_02',
 'driver3_lap_02',
 'driver4_lap_02',
 'driver1_lap_03',
 'driver2_lap_03',
 'driver3_lap_03',
 'driver4_lap_03',
 'driver1_lap_04',
 'driver2_lap_04',
 'driver3_lap_04',
 'driver4_lap_04',
 'driver1_lap_05',
 'driver2_lap_05',
 'driver3_lap_05',
 'driver4_lap_05',
 'driver1_lap_06',
 'driver2_lap_06',
 'driver3_lap_06',
 'driver4_lap_06',
 'driver1_lap_07',
 'driver2_lap_07',
 'driver3_lap_07',
 'driver4_lap_07',
 'driver1_lap_08',
 'driver2_lap_08',
 'driver3_lap_08',
 'driver4_lap_08',
 'driver1_lap_09',
 'driver2_lap_09',
 'driver3_lap_09',
 'driver4_lap_09',
 'driver1_lap_10',
 'driver2_lap_10',
 'driver3_lap_10',
 'driver4_lap_10',
 'driver1_y',
 'driver2_y',
 'driver3_y',
 'driver4_y']

In [296]:
['_dr'.join(map(str, col)).strip() for col in ptr.columns.values]

['copy_id_dr',
 'lap_01_dr1',
 'lap_01_dr2',
 'lap_01_dr3',
 'lap_01_dr4',
 'lap_02_dr1',
 'lap_02_dr2',
 'lap_02_dr3',
 'lap_02_dr4',
 'lap_03_dr1',
 'lap_03_dr2',
 'lap_03_dr3',
 'lap_03_dr4',
 'lap_04_dr1',
 'lap_04_dr2',
 'lap_04_dr3',
 'lap_04_dr4',
 'lap_05_dr1',
 'lap_05_dr2',
 'lap_05_dr3',
 'lap_05_dr4',
 'lap_06_dr1',
 'lap_06_dr2',
 'lap_06_dr3',
 'lap_06_dr4',
 'lap_07_dr1',
 'lap_07_dr2',
 'lap_07_dr3',
 'lap_07_dr4',
 'lap_08_dr1',
 'lap_08_dr2',
 'lap_08_dr3',
 'lap_08_dr4',
 'lap_09_dr1',
 'lap_09_dr2',
 'lap_09_dr3',
 'lap_09_dr4',
 'lap_10_dr1',
 'lap_10_dr2',
 'lap_10_dr3',
 'lap_10_dr4',
 'y_dr1',
 'y_dr2',
 'y_dr3',
 'y_dr4']

In [184]:
laps_to_consider = [1,2,3,4,5,6,7,8,9,10]
# get laptimes (note: all races in laptimes_raw_df are in races dataframe)
laptimes = laptimes_raw_df[laptimes_raw_df['raceId'].isin(races['raceId'].unique())].reset_index(drop=True)[['raceId','driverId','lap','position','milliseconds']]
print('max number drivers per race', laptimes.groupby(['raceId'])['driverId'].nunique().max())

# calculate driver finishing times
driver_finishing_times = laptimes.groupby(['raceId','driverId'])['milliseconds'].sum().reset_index(name='finishing_time') # todo: fix where driver doesn't finish

# filter data so that it corresponds to desired feature set
laptimes = laptimes[laptimes['lap'].isin(laps_to_consider)].reset_index(drop=True)

max number drivers per race 24


In [ ]:
train

In [191]:
metric = 'position'
if metric == 'milliseconds':
    model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=0)  # fill missing values with 0
elif metric == 'position':
    model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=25)  # fill missing values with 25
# else:
#     return None

#model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values='position', fill_value=25)  # fill missing values with 25
model_data.reset_index(inplace=True)
model_data.head(30)


positions = races[['raceId','driverId','positionText']].drop_duplicates().reset_index(drop=True)
#display(positions['positionText'].value_counts())
model_position_text_map = {'Retired':'25','Disqualified':'25', 'not classified':'25','Excluded':'25'}

positions['Y'] = positions['positionText'].replace(model_position_text_map)
positions.drop(['positionText'], axis=1, inplace=True)
prev_len = model_data.shape[0]
model_data = model_data.merge(positions, on=['raceId','driverId'], how='left')
#display(model_data['Y'].value_counts())

assert model_data.shape[0] == prev_len, 'duplicates formed during merge'
assert model_data.groupby(['raceId','driverId'])['Y'].nunique().max() == 1, 'each driver should have one finishing position'
assert model_data['Y'].isin(['Withdrawn','failed to qualify']).sum() == 0, 'should not have any drivers who withdrew or failed to qualify'
model_data['Y'] = model_data['Y'].astype(int)
assert model_data.groupby(['raceId'])['Y'].min().max() == 1, 'races missing a winner'
assert model_data.groupby(['raceId'])['Y'].min().min() == 1, 'positions < 1'
assert model_data.groupby(['raceId'])['Y'].max().max() <= 25, 'race has position > 24'
assert model_data.isna().sum().sum() ==0

model_data = model_data.groupby('raceId').apply(pad_race).reset_index(drop=True)

# replace races with 3 random-order copies
model_data = model_data.groupby('raceId', group_keys=False).apply(randomize_race, (3)).reset_index(drop=True)


# binary_y = True
# if binary_y:
#     model_data['Y'] = model_data['Y'].apply(lambda x: 0 if x != 1 else 1)

model_data = model_data.groupby('raceId', group_keys=False).apply(randomize_race, (3)).reset_index(drop=True)

train_df, valid_df, test_df = train_test_split_by_id(model_data, 'copy_id')

train_df_mf = pivot_model_data(train_df).set_index('copy_id')
valid_df_mf = pivot_model_data(valid_df).set_index('copy_id')
test_df_mf = pivot_model_data(test_df).set_index('copy_id')


#train_X, train_y = train_df

# make folder if doesn't exist
if not os.path.exists('Data/RaceMultiOutputModel/'):
    os.makedirs('Data/RaceMultiOutputModel/')

# train_df_mf.to_csv('Data/RaceMultiOutputModel/train_X.csv', index=False)
# valid_df_mf.to_csv('Data/RaceMultiOutputModel/valid_X.csv', index=False)
# test_df_mf.to_csv('Data/RaceMultiOutputModel/test_X.csv', index=False)

In [205]:
train_df.sort_values(['copy_id']).head(24)

,raceId,driverId,1,2,3,4,5,6,7,8,9,10,Y,copy_id
87749,1000,830,5,5,5,5,5,25,25,25,25,25,0,1000_1
87748,1000,843,9,9,9,9,9,8,8,8,8,8,0,1000_1
87747,1000,817,16,16,15,14,14,12,12,12,11,11,0,1000_1
87746,1000,4,11,11,11,11,11,10,11,11,12,12,0,1000_1
87745,1000,845,17,17,17,17,17,16,16,16,16,16,0,1000_1
87744,1000,8,4,4,4,4,4,4,4,4,4,4,0,1000_1
87743,1000,842,6,6,6,6,6,5,5,5,5,5,0,1000_1
87750,1000,817,16,16,15,14,14,12,12,12,11,11,0,1000_1
87742,1000,845,17,17,17,17,17,16,16,16,16,16,0,1000_1
87739,1000,4,11,11,11,11,11,10,11,11,12,12,0,1000_1


In [150]:

# Assuming `df` is your DataFrame as shown in the screenshot

# Filter out only the lap columns and the 'Y' column
laps_and_y_cols = [col for col in train_df.columns if str(col).isdigit() or col == 'Y']

# Use pivot_table to pivot the entire DataFrame at once
pivot = train_df.pivot_table(index='copy_id', 
                       columns=['driverId'], 
                       values=laps_and_y_cols, fill_value='FILL')

# Flatten the MultiIndex in columns
pivot.columns = [f'driver_id{driver_id}_{col}pos' for col, driver_id in pivot.columns]

# Reset the index so 'copy_id' is a column again
pivot.reset_index(inplace=True)

pivot


copy_id driver_id0_1pos driver_id1_1pos driver_id2_1pos driver_id3_1pos  \
0     1000_1             0.0             1.0            FILL            FILL   
1     1000_3             0.0             1.0            FILL            FILL   
2     1001_1             0.0             2.0            FILL            FILL   
3     1001_2             0.0             2.0            FILL            FILL   
4     1002_1             0.0             2.0            FILL            FILL   
...      ...             ...             ...             ...             ...   
1219    99_1             0.0            FILL            17.0            FILL   
1220    99_2             0.0            FILL            17.0            FILL   
1221    99_3             0.0            FILL            17.0            FILL   
1222     9_1             0.0            20.0            12.0             9.0   
1223     9_2             0.0            20.0            12.0             9.0   

     driver_id4_1pos driver_id5_1pos driver_id6_1pos driver_id7_1pos  \
0               11.0            FILL            FILL            FILL   
1               11.0            FILL            FILL            FILL   
2               FILL            FILL            FILL            FILL   
3               FILL            FILL            FILL            FILL   
4               10.0            FILL            FILL            FILL   
...              ...             ...             ...             ...   
1219             1.0            FILL            FILL            FILL   
1220             1.0            FILL            FILL            FILL   
1221             1.0            FILL            FILL            FILL   
1222            11.0             3.0            19.0            17.0   
1223            11.0             3.0            19.0            17.0   

     driver_id8_1pos driver_id9_1pos driver_id10_1pos driver_id11_1pos  \
0                4.0            FILL             FILL             FILL   
1                4.0            FILL             FILL             FILL   
2               17.0            FILL             FILL             FILL   
3               17.0            FILL             FILL             FILL   
4                1.0            FILL             FILL             FILL   
...              ...             ...              ...              ...   
1219             7.0            FILL             FILL              8.0   
1220             7.0            FILL             FILL              8.0   
1221             7.0            FILL             FILL              8.0   
1222             7.0            10.0             18.0             FILL   
1223             7.0            10.0             18.0             FILL   

     driver_id12_1pos driver_id13_1pos driver_id14_1pos driver_id15_1pos  \
0                FILL             FILL             FILL             FILL   
1                FILL             FILL             FILL             FILL   
2                FILL             FILL             FILL             FILL   
3                FILL             FILL             FILL             FILL   
4                FILL             FILL             FILL             FILL   
...               ...              ...              ...              ...   
1219             FILL             14.0              5.0              3.0   
1220             FILL             14.0              5.0              3.0   
1221             FILL             14.0              5.0              3.0   
1222             15.0              4.0             FILL             16.0   
1223             15.0              4.0             FILL             16.0   

     driver_id16_1pos driver_id17_1pos driver_id18_1pos driver_id19_1pos  \
0                FILL             FILL             FILL             FILL   
1                FILL             FILL             FILL             FILL   
2                FILL             FILL             FILL             FILL   
3                FILL             FILL             FILL             FILL 

In [182]:
def pivot_model_data(df):
    df = df.sort_values(by=['copy_id', 'driverId'])

    # Create 'model_driverId' within each 'copy_id' group
    df['model_driverId'] = df.groupby('copy_id').cumcount() + 1


    # value columns
    laps_and_y_cols = [col for col in df.columns if str(col).isdigit() or col == 'Y']

    pivot = df.pivot_table(index='copy_id', 
                        columns=['model_driverId'], 
                        values=laps_and_y_cols, fill_value=np.nan)

    # Flatten the MultiIndex in columns
    pivot.columns = [f'model_driver{model_driver_id}_{lap_id}pos' if lap_id else 'copy_id' 
                    for model_driver_id, lap_id in pivot.columns]

    pivot.reset_index(inplace=True, drop=False)


In [179]:
df = train_df.sort_values(by=['copy_id', 'driverId'])

# Create 'model_driverId' within each 'copy_id' group
df['model_driverId'] = df.groupby('copy_id').cumcount() + 1


# value columns
laps_and_y_cols = [col for col in df.columns if str(col).isdigit() or col == 'Y']

pivot = df.pivot_table(index='copy_id', 
                       columns=['model_driverId'], 
                       values=laps_and_y_cols, fill_value=np.nan)

# Flatten the MultiIndex in columns
pivot.columns = [f'model_driver{model_driver_id}_{lap_id}pos' if lap_id else 'copy_id' 
                 for model_driver_id, lap_id in pivot.columns]

pivot.reset_index(inplace=True, drop=False)

pivot.iloc[:,-24:]



,model_driverY_1pos,model_driverY_2pos,model_driverY_3pos,model_driverY_4pos,model_driverY_5pos,model_driverY_6pos,model_driverY_7pos,model_driverY_8pos,model_driverY_9pos,model_driverY_10pos,model_driverY_11pos,model_driverY_12pos,model_driverY_13pos,model_driverY_14pos,model_driverY_15pos,model_driverY_16pos,model_driverY_17pos,model_driverY_18pos,model_driverY_19pos,model_driverY_20pos,model_driverY_21pos,model_driverY_22pos,model_driverY_23pos,model_driverY_24pos
0,25,25,25,25,25,1,8,3,2,10,12,14,4,5,7,15,25,9,25,13,17,6,11,16
1,25,25,25,25,25,1,8,3,2,10,12,14,4,5,7,15,25,9,25,13,17,6,11,16
2,25,25,25,25,25,25,25,2,25,1,7,5,25,4,8,10,3,11,15,6,13,9,14,12
3,25,25,25,25,25,25,25,2,25,1,7,5,25,4,8,10,3,11,15,6,13,9,14,12
4,25,25,25,25,25,1,25,2,4,25,13,7,25,3,16,15,5,8,12,6,9,14,11,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,25,25,25,25,16,2,7,25,13,6,4,9,5,12,3,1,8,11,14,15,17,18,25,10
1220,25,25,25,25,16,2,7,25,13,6,4,9,5,12,3,1,8,11,14,15,17,18,25,10
1221,25,25,25,25,16,2,7,25,13,6,4,9,5,12,3,1,8,11,14,15,17,18,25,10
1222,25,25,25,25,18,10,4,7,8,12,25,25,14,9,13,3,17,15,1,5,2,11,6,16


In [181]:
df[df['copy_id']=='1000_1'][['Y']].T

,29234,29236,29239,29250,29253,29252,29237,29245,29251,29244,29246,29242,29249,29241,29248,29233,29232,29255,29238,29235,29254,29240,29243,29247
Y,25,25,25,25,25,1,8,3,2,10,12,14,4,5,7,15,25,9,25,13,17,6,11,16


In [171]:
train_df[['Y']].head(24).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Y,3,25,25,1,25,10,12,25,11,4,15,2,25,6,8,7,25,14,25,5,25,25,13,9


In [147]:
u = train_df.pivot_table(index=['copy_id'], columns=['driverId','lap'],  fill_value=np.nan)


KeyError: 'lap'

In [112]:
test_df.shape

(3672, 14)

In [111]:
valid_df.shape

(3672, 14)

In [64]:
# # Assuming `df` is your original DataFrame
# # First, let's set the index of the DataFrame to be ['raceId', 'driverId'] if it's not already
# df = model_data.set_index(['raceId', 'driverId'])

# # Now, let's create the 2D DataFrame
# flattened_df = df.groupby('raceId').apply(lambda x: x.unstack().values.flatten())

# # If you want to reset the index and have raceId as a column
# flattened_df = flattened_df.reset_index()

# # Rename columns to reflect that they are flattened laps
# flattened_df.columns = ['raceId'] + ['lap_{}'.format(i) for i in range(1, flattened_df.shape[1])]

# flattened_df.head()

ValueError: Index contains duplicate entries, cannot reshape

In [158]:
LAPTIMES = laptimes

In [159]:
# Reshape data for model input
X = laptimes.pivot_table(index='raceId', columns=['driver_race_model_pos', 'lap'], values='milliseconds', fill_value=0)  # fill missing values with 0
# Flatten the multi-level columns after pivot
X.columns = ['driver{}_lap{}'.format(*col) for col in X.columns.values]
# Reset index to turn the raceId back into a column
X = X.reset_index()
X = X.merge(races[['raceId','date']].drop_duplicates(), on='raceId', how='left')
X.sort_values(by='date', inplace=True)
X_train = X.copy(deep=True)[X['date'] < datetime.datetime(2021, 1, 1)]
print(X_train.shape)
X_test = X.copy(deep=True)[X['date'] > datetime.datetime(2021, 1, 1)]
print(X_test.shape)
X.drop(['date'], axis=1, inplace=True)
X_train.drop(['date'], axis=1, inplace=True)
X_test.drop(['date'], axis=1, inplace=True)

X_train.to_csv('Data/RaceMultiOutputModel/time_X_train.csv', index=False)
X_test.to_csv('Data/RaceMultiOutputModel/time_X_test.csv', index=False)

#X_train = X[X['date'] < datetime.datetime(2021, 1, 1)]
#X.to_csv('Data/RaceMultiOutputModel/X.csv', index=False)


# Reshape data for model input
X = laptimes.pivot_table(index='raceId', columns=['driver_race_model_pos', 'lap'], values='position', fill_value=25)  # fill missing values with 0
# Flatten the multi-level columns after pivot
X.columns = ['driver{}_lap{}'.format(*col) for col in X.columns.values]
# Reset index to turn the raceId back into a column
X = X.reset_index()
X = X.merge(races[['raceId','date']].drop_duplicates(), on='raceId', how='left')
X.sort_values(by='date', inplace=True)
X_train = X.copy(deep=True)[X['date'] < datetime.datetime(2021, 1, 1)]
print(X_train.shape)
X_test = X.copy(deep=True)[X['date'] > datetime.datetime(2021, 1, 1)]
print(X_test.shape)
X.drop(['date'], axis=1, inplace=True)
X_train.drop(['date'], axis=1, inplace=True)
X_test.drop(['date'], axis=1, inplace=True)

X_train.to_csv('Data/RaceMultiOutputModel/pos_X_train.csv', index=False)
X_test.to_csv('Data/RaceMultiOutputModel/pos_X_test.csv', index=False)

#X_train = X[X['date'] < datetime.datetime(2021, 1, 1)]
#X.to_csv('Data/RaceMultiOutputModel/X.csv', index=False)

(454, 242)
(56, 242)
(454, 242)
(56, 242)


In [160]:
display(X_test[X_test['raceId']==1054])

,raceId,driver1_lap1,driver1_lap2,driver1_lap3,driver1_lap4,driver1_lap5,driver1_lap6,driver1_lap7,driver1_lap8,driver1_lap9,driver1_lap10,driver2_lap1,driver2_lap2,driver2_lap3,driver2_lap4,driver2_lap5,driver2_lap6,driver2_lap7,driver2_lap8,driver2_lap9,driver2_lap10,driver3_lap1,driver3_lap2,driver3_lap3,driver3_lap4,driver3_lap5,driver3_lap6,driver3_lap7,driver3_lap8,driver3_lap9,driver3_lap10,driver4_lap1,driver4_lap2,driver4_lap3,driver4_lap4,driver4_lap5,driver4_lap6,driver4_lap7,driver4_lap8,driver4_lap9,driver4_lap10,driver5_lap1,driver5_lap2,driver5_lap3,driver5_lap4,driver5_lap5,driver5_lap6,driver5_lap7,driver5_lap8,driver5_lap9,driver5_lap10,driver6_lap1,driver6_lap2,driver6_lap3,driver6_lap4,driver6_lap5,driver6_lap6,driver6_lap7,driver6_lap8,driver6_lap9,driver6_lap10,driver7_lap1,driver7_lap2,driver7_lap3,driver7_lap4,driver7_lap5,driver7_lap6,driver7_lap7,driver7_lap8,driver7_lap9,driver7_lap10,driver8_lap1,driver8_lap2,driver8_lap3,driver8_lap4,driver8_lap5,driver8_lap6,driver8_lap7,driver8_lap8,driver8_lap9,driver8_lap10,driver9_lap1,driver9_lap2,driver9_lap3,driver9_lap4,driver9_lap5,driver9_lap6,driver9_lap7,driver9_lap8,driver9_lap9,driver9_lap10,driver10_lap1,driver10_lap2,driver10_lap3,driver10_lap4,driver10_lap5,driver10_lap6,driver10_lap7,driver10_lap8,driver10_lap9,driver10_lap10,driver11_lap1,driver11_lap2,driver11_lap3,driver11_lap4,driver11_lap5,driver11_lap6,driver11_lap7,driver11_lap8,driver11_lap9,driver11_lap10,driver12_lap1,driver12_lap2,driver12_lap3,driver12_lap4,driver12_lap5,driver12_lap6,driver12_lap7,driver12_lap8,driver12_lap9,driver12_lap10,driver13_lap1,driver13_lap2,driver13_lap3,driver13_lap4,driver13_lap5,driver13_lap6,driver13_lap7,driver13_lap8,driver13_lap9,driver13_lap10,driver14_lap1,driver14_lap2,driver14_lap3,driver14_lap4,driver14_lap5,driver14_lap6,driver14_lap7,driver14_lap8,driver14_lap9,driver14_lap10,driver15_lap1,driver15_lap2,driver15_lap3,driver15_lap4,driver15_lap5,driver15_lap6,driver15_lap7,driver15_lap8,driver15_lap9,driver15_lap10,driver16_lap1,driver16_lap2,driver16_lap3,driver16_lap4,driver16_lap5,driver16_lap6,driver16_lap7,driver16_lap8,driver16_lap9,driver16_lap10,driver17_lap1,driver17_lap2,driver17_lap3,driver17_lap4,driver17_lap5,driver17_lap6,driver17_lap7,driver17_lap8,driver17_lap9,driver17_lap10,driver18_lap1,driver18_lap2,driver18_lap3,driver18_lap4,driver18_lap5,driver18_lap6,driver18_lap7,driver18_lap8,driver18_lap9,driver18_lap10,driver19_lap1,driver19_lap2,driver19_lap3,driver19_lap4,driver19_lap5,driver19_lap6,driver19_lap7,driver19_lap8,driver19_lap9,driver19_lap10,driver20_lap1,driver20_lap2,driver20_lap3,driver20_lap4,driver20_lap5,driver20_lap6,driver20_lap7,driver20_lap8,driver20_lap9,driver20_lap10,driver21_lap1,driver21_lap2,driver21_lap3,driver21_lap4,driver21_lap5,driver21_lap6,driver21_lap7,driver21_lap8,driver21_lap9,driver21_lap10,driver22_lap1,driver22_lap2,driver22_lap3,driver22_lap4,driver22_lap5,driver22_lap6,driver22_lap7,driver22_lap8,driver22_lap9,driver22_lap10,driver23_lap1,driver23_lap2,driver23_lap3,driver23_lap4,driver23_lap5,driver23_lap6,driver23_lap7,driver23_lap8,driver23_lap9,driver23_lap10,driver24_lap1,driver24_lap2,driver24_lap3,driver24_lap4,driver24_lap5,driver24_lap6,driver24_lap7,driver24_lap8,driver24_lap9,driver24_lap10
457,1054,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,2,2,2,2,4,4,4,4,4,4,6,6,6,6,5,5,5,5,5,5,5,5,5,5,6,6,6,6,6,6,4,4,4,4,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,8,7,7,7,7,9,9,9,9,9,9,9,9,9,9,10,10,10,10,10,10,10,10,10,10,11,11,11,11,11,11,14,15,15,15,12,12,12,12,12,12,12,12,12,12,13,25,25,25,25,25,25,25,25,25,14,13,13,13,13,13,11,11,11,11,15,14,14,14,14,14,13,13,13,13,16,15,15,15,15,15,16,16,16,16,17,16,16,16,16,16,15,14,14,14,18,17,17,17,17,17,17,17,17,17,19,18,18,18,18,18,19,18,18,18,20,19,19,19,19,19,18,19,19,19,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25


In [161]:
# Y = laptimes[['raceId','driverId']].drop_duplicates().reset_index(drop=True)
# assert races.groupby(['raceId','driverId'])['position'].nunique().max() == 1, 'There should only be 1 position reported for each driver in each race'
# print(Y.shape)
# print(raceId_driverId_order_filled.shape)

# Process Y
# start with raceId_driver_Id_order_filled (because some drivers will be in the races dataframe but not in the laptimes dataframe due to Withdrawal or failing to qualify)
driver_model_order_finishing_positions = raceId_driverId_order_filled.copy(deep=True)
# merge with races -- with raceId_driver_Id_order_filled on left -- to get positionText
prev_len = driver_model_order_finishing_positions.shape[0]
driver_model_order_finishing_positions = raceId_driverId_order_filled.merge(races[['raceId','driverId','positionText']], on=['raceId','driverId'], how='left')
assert driver_model_order_finishing_positions.shape[0] == prev_len, 'duplicates formed during merge'
assert driver_model_order_finishing_positions['positionText'].isin(['Withdrawn','failed to qualify']).sum() == 0, 'should not have any drivers who withdrew or failed to qualify'
driver_model_order_finishing_positions.sort_values(by=['raceId','driver_race_model_pos']).head(25)

# print(driver_model_order_finishing_positions['positionText'].unique())
display(driver_model_order_finishing_positions['positionText'].value_counts())
model_position_text_map = {'Retired':'25','Disqualified':'25', 'not classified':'25','Excluded':'25'}

driver_model_order_finishing_positions['Y'] = driver_model_order_finishing_positions['positionText'].replace(model_position_text_map)
driver_model_order_finishing_positions['Y'].fillna('25', inplace=True) # note: these NaNs are only there because we are creating dummy drivers for races with less than 24 drivers
driver_model_order_finishing_positions['Y'] = driver_model_order_finishing_positions['Y'].astype(int)
driver_model_order_finishing_positions.drop(['positionText'], axis=1, inplace=True)
assert driver_model_order_finishing_positions.isna().sum().sum() ==0 

# test case: driver 4, race 2 only has lap times for 1 lap
DRIVER_MODEL_ORDER_FINISHING_POSITIONS = driver_model_order_finishing_positions


Retired           2211
6                  510
2                  510
1                  510
5                  510
3                  510
4                  510
7                  508
8                  506
9                  502
10                 494
11                 477
12                 450
13                 425
14                 388
15                 359
16                 309
17                 252
18                 173
19                 107
20                  60
Disqualified        40
21                  30
22                  16
not classified      10
23                   5
24                   1
Excluded             1
Name: positionText, dtype: int64

In [35]:
Y = driver_model_order_finishing_positions.pivot_table(index='raceId',columns='driver_race_model_pos',values='Y')
Y = Y.merge(races[['raceId','date']].drop_duplicates(), on='raceId', how='left')

# X = X.merge(races[['raceId','date']].drop_duplicates(), on='raceId', how='left')
Y.sort_values(by='date', inplace=True)
Y_train = Y.copy(deep=True)[Y['date'] < datetime.datetime(2021, 1, 1)]
print(Y_train.shape)
Y_test = Y.copy(deep=True)[Y['date'] > datetime.datetime(2021, 1, 1)]
print(Y_test.shape)
Y.drop(['date'], axis=1, inplace=True)
Y_train.drop(['date'], axis=1, inplace=True)
Y_test.drop(['date'], axis=1, inplace=True)

Y_train.to_csv('Data/RaceMultiOutputModel/Y_train.csv', index=False)
Y_test.to_csv('Data/RaceMultiOutputModel/Y_test.csv', index=False)

# Y.to_csv('Data/RaceMultiOutputModel/Y.csv')

(454, 26)
(56, 26)


In [38]:
Y_race1054 = Y_test[Y_test['raceId']==1054].copy(deep=True)
Y_race1054

,raceId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
457,1054,3,1,2,11,4,5,7,6,10,13,16,12,25,9,8,15,14,18,17,19,25,25,25,25


In [45]:
races

,resultId,raceId,driverId,grid,position,positionText,points,Race,round,date,url,Month,Year,Name,status,Car,Circuit,location,country,alt,lat,lng,nlaps
0,1,18,1,1,1,1,10.0,Australian Grand Prix,1,2008-03-16,http://en.wikipedia.org/wiki/2008_Australian_G...,3,2008,Lewis Hamilton,Finished,McLaren,Albert Park Grand Prix Circuit,Melbourne,Australia,10,-37.8497,144.96800,58
1,2,18,2,5,2,2,8.0,Australian Grand Prix,1,2008-03-16,http://en.wikipedia.org/wiki/2008_Australian_G...,3,2008,Nick Heidfeld,Finished,BMW Sauber,Albert Park Grand Prix Circuit,Melbourne,Australia,10,-37.8497,144.96800,58
2,3,18,3,7,3,3,6.0,Australian Grand Prix,1,2008-03-16,http://en.wikipedia.org/wiki/2008_Australian_G...,3,2008,Nico Rosberg,Finished,Williams,Albert Park Grand Prix Circuit,Melbourne,Australia,10,-37.8497,144.96800,58
3,4,18,4,11,4,4,5.0,Australian Grand Prix,1,2008-03-16,http://en.wikipedia.org/wiki/2008_Australian_G...,3,2008,Fernando Alonso,Finished,Renault,Albert Park Grand Prix Circuit,Melbourne,Australia,10,-37.8497,144.96800,58
4,5,18,5,3,5,5,4.0,Australian Grand Prix,1,2008-03-16,http://en.wikipedia.org/wiki/2008_Australian_G...,3,2008,Heikki Kovalainen,Finished,McLaren,Albert Park Grand Prix Circuit,Melbourne,Australia,10,-37.8497,144.96800,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22941,26081,1110,817,19,16,16,0.0,Belgian Grand Prix,12,2023-07-30,https://en.wikipedia.org/wiki/2023_Belgian_Gra...,7,2023,Daniel Ricciardo,Finished,AlphaTauri,Circuit de Spa-Francorchamps,Spa,Belgium,401,50.4372,5.97139,44
22942,26082,1110,858,18,17,17,0.0,Belgian Grand Prix,12,2023-07-30,https://en.wikipedia.org/wiki/2023_Belgian_Gra...,7,2023,Logan Sargeant,Finished,Williams,Circuit de Spa-Francorchamps,Spa,Belgium,401,50.4372,5.97139,44
22943,26083,1110,807,0,18,18,0.0,Belgian Grand Prix,12,2023-07-30,https://en.wikipedia.org/wiki/2023_Belgian_Gra...,7,2023,Nico Hülkenberg,Finished,Haas F1 Team,Circuit de Spa-Francorchamps,Spa,Belgium,401,50.4372,5.97139,44
22944,26084,1110,832,4,NaN,Retired,0.0,Belgian Grand Prix,12,2023-07-30,https://en.wikipedia.org/wiki/2023_Belgian_Gra...,7,2023,Carlos Sainz,Collision damage,Ferrari,Circuit de Spa-Francorchamps,Spa,Belgium,401,50.4372,5.97139,44


In [51]:
model_finishing_pos = Y_race1054[1].values[0]
races[(races['raceId']==1054) & (races['position']=='3')]

,resultId,raceId,driverId,grid,position,positionText,points,Race,round,date,url,Month,Year,Name,status,Car,Circuit,location,country,alt,lat,lng,nlaps
21868,25008,1054,822,1,3,3,16.0,Portuguese Grand Prix,3,2021-05-02,http://en.wikipedia.org/wiki/2021_Portuguese_G...,5,2021,Valtteri Bottas,Finished,Mercedes,Autódromo Internacional do Algarve,Portimão,Portugal,108,37.227,-8.6267,66


In [62]:
for i in range(1,25):
    model_race_pos = i
    model_finishing_pos = Y_race1054[i].values[0]

    driverId_from_model = raceId_driverId_order_filled[(raceId_driverId_order_filled['raceId']==1054) & (raceId_driverId_order_filled['driver_race_model_pos']==i)]['driverId'].values[0]
    try:
        driverid_at_pos = races[(races['raceId']==1054) & (races['position']==str(model_finishing_pos))]['driverId'].values[0]
        assert driverId_from_model == driverid_at_pos, 'driverId_from_model != driver_at_pos'
    except:
        driverid_at_pos = 'no driver at pos'
    
    print('Model Order', i, 'Finishing Position', Y_race1054[i].values[0], 'driverId', driverId_from_model, 'driver at pos', driverid_at_pos)

Model Order 1 Finishing Position 3 driverId 822 driver at pos 822
Model Order 2 Finishing Position 1 driverId 1 driver at pos 1
Model Order 3 Finishing Position 2 driverId 830 driver at pos 830
Model Order 4 Finishing Position 11 driverId 832 driver at pos 832
Model Order 5 Finishing Position 4 driverId 815 driver at pos 815
Model Order 6 Finishing Position 5 driverId 846 driver at pos 846
Model Order 7 Finishing Position 7 driverId 839 driver at pos 839
Model Order 8 Finishing Position 6 driverId 844 driver at pos 844
Model Order 9 Finishing Position 10 driverId 842 driver at pos 842
Model Order 10 Finishing Position 13 driverId 20 driver at pos 20
Model Order 11 Finishing Position 16 driverId 847 driver at pos 847
Model Order 12 Finishing Position 12 driverId 841 driver at pos 841
Model Order 13 Finishing Position 25 driverId 8 driver at pos no driver at pos
Model Order 14 Finishing Position 9 driverId 817 driver at pos 817
Model Order 15 Finishing Position 8 driverId 4 driver at pos

In [37]:
assert (X_train['raceId'] != Y_train['raceId']).sum()==0
assert (X_test['raceId'] != Y_test['raceId']).sum()==0